In [65]:
from transformers import BartTokenizer, PegasusTokenizer
from transformers import BartForConditionalGeneration, PegasusForConditionalGeneration
import os
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
from gensim.test.utils import common_corpus, common_dictionary
from gensim.models.coherencemodel import CoherenceModel
import gensim.corpora as corpora
from nltk.tokenize import word_tokenize
from rouge_score import rouge_scorer

In [181]:
IS_CNNDM = True

max_length = 1024 if IS_CNNDM else 512

if IS_CNNDM:
    model = BartForConditionalGeneration.from_pretrained('Yale-LILY/brio-cnndm-uncased')
    tokenizer = BartTokenizer.from_pretrained('Yale-LILY/brio-cnndm-uncased')
else:
    model = PegasusForConditionalGeneration.from_pretrained('Yale-LILY/brio-xsum-cased')
    tokenizer = PegasusTokenizer.from_pretrained('Yale-LILY/brio-xsum-cased')

Downloading:  10%|▉         | 97.1M/971M [8:29:54<76:29:59, 3.33kB/s]


In [171]:
#BART

model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")

In [67]:
from sentence_transformers import SentenceTransformer, util
smodel = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [165]:
data = pd.read_csv("SEBI Summaries.csv")
data.head()

,File no.,Fact,Violations,Penalty,Defense,AO,Investor_Summary,Defense_Summary,AO_Summary
0,1598615418309_1,"Chromatic India Ltd. was penalised 10,00,00,00...","Section 12A(a), (b), (c) of the SEBI Act read ...","3,00,000/- (Rupees Three lakh) on the noticee ...",The primary contention of the noticee w.r.t th...,Order of the SAT in SEBI vs. Kalidas Dutta; Ap...,"Chromatic India Ltd. was penalised 10,00,00,00...","Chromatic India Ltd. was penalised 10,00,00,00...","Chromatic India Ltd. was penalised 10,00,00,00..."
1,1598615418309_2,GRM Securities Pvt. Ltd. contributed 7.32% of ...,"clauses A(1), A(2) and A(4) of Code of Conduct...","4,00,000/- (Rupees Four Lakh only) on the Noti...",The Adjudicating Officer dismissed the claim ...,SEBI Vs. Shri Ram Mutual Fund [2006] 68 SCL 21...,GRM Securities Pvt. Ltd. contributed 7.32% of ...,GRM Securities Pvt. Ltd. contributed 7.32% of ...,GRM Securities Pvt. Ltd. contributed 7.32% of ...
2,1599654002962_1,Ms. Rekha Jain (noticee) failed to make disclo...,regulation 13 (4A) read with 13(5) of PIT Regu...,"failure to disclose acquisition of 45,000 shar...",the Noticee has placed reliance upon the discl...,The Noticee has also furnished the Merger Orde...,Ms. Rekha Jain (noticee) failed to make disclo...,Ms. Rekha Jain (noticee) failed to make disclo...,Ms. Rekha Jain (noticee) failed to make disclo...
3,1599654002962_2,Sanket Rakesh Jain (Noticee) failed to disclos...,regulation 13(2A) of the PIT regulations and r...,"failed to disclose his acquisition of 1,24,475...",the Noticee has placed reliance upon the discl...,The Noticee has also furnished the Merger Orde...,Sanket Rakesh Jain (Noticee) failed to disclos...,Sanket Rakesh Jain (Noticee) failed to disclos...,Sanket Rakesh Jain (Noticee) failed to disclos...
4,1602679805625_2,"From 2011 to 2014, SEBI directed all companies...","Rs. 1,00,000/- under Section 15 A(a) of the SE...",After taking into consideration all the facts ...,Noticee has not availed the opportunity of per...,"Thus, the aforesaid non-compliance to SEBI cir...","From 2011 to 2014, SEBI directed all companies...","From 2011 to 2014, SEBI directed all companies...","From 2011 to 2014, SEBI directed all companies..."


In [178]:
loc = ["investor", "defense", "ao"]

count = 0

f1 = open("pipe_brio_out.txt", "w+", encoding="utf-8")
f2 = open("brio_out.txt", "w+", encoding="utf-8")

for i in loc:
    for j in os.listdir("./pipe_annot_out/"+i+"/"):
        with open("./pipe_annot_out/"+i+"/"+j, encoding="utf-8") as f3:
            text = f3.read()
            f1.write(" ".join(text.split("\n")))
            f1.write("\n")
            f3.close()
        with open("./annot_out/"+i+"/"+j, encoding="utf-8") as f3:
            text = f3.read()
            f2.write(" ".join(text.split("\n")))
            f2.write("\n")
            f3.close()
f1.close()
f2.close()

In [179]:
source = "The Securities and Exchange Board of India (hereinafter referred to as ‘SEBI’) investigated into various price sensitive disclosure made by M s Suzlon Energy Limited (hereinafter referred to as ‘Suzlon the company’) for the period of November 3, 2009 and December 2, 2009. During the course of investigation it was observed that Suzlon had amended its code of internal procedures and conduct for prevention of insider trading for listed companies (“Code of Conduct”) in line with amendments in SEBI (Prohibition of Insider Trading) Regulations, 1992 (hereinafter referred to as “PIT Regulations”). Adjudication Order in respect of Vinod R. Tanti in the matter of Suzlon Energy Limited PIT Regulations notified on November 19, 2008 only on February 4th, 2011, i.e. after a delay of more than 2 years. It was alleged that the Code of Conduct was to be implemented under the overall supervision of the Board of Directors of Suzlon (hereinafter referred to as “the Board”). Shri Vinod Tanti (hereinafter referred to as “the Noticee”), became a member of the Board between November 19, 2008 and February 4, 2011. As a member of the Board, the Noticee was also allegedly responsible for the delay in amendment and implementation of the Code of Conduct. Accordingly, Suzlon was required to amend its code of conduct in line with the amendments in PIT Regulations. However, Suzlon implemented the amended Code of Conduct only on February 4th, 2011, which is more than 2 years after the notification. This huge delay occurred inspite of Suzlon’s Board of Directors being aware about the changes in PIT Regulations. The Board was informed about the changes in the PIT regulations in the meeting dated July 31, 2009 by Mr. Sundar Rajagopalan, who was the President (Legal) and General Counsel of Suzlon. The Board unduly delayed the implementation of revised Code of Conduct by deferring the matter when it was listed as agenda item in the Board meetings dated May 29, 2010 and August 13, 2010 and the code was subsequently passed in the board meeting dated February 4, 2011. It is observed from the meeting wise constitution of the Board that the Noticee was No a member of the Board till the meeting on August 13, 2010. The Noticee was made a director between August 13, 2010 and February 4, 2011. The Noticee attended his first board meeting after appointment on February 4, 2011. This was the meeting in which the code of conduct was approved by the Board."
summ = "SEBI investigated Suzlon Energy Limited for price sensitive disclosure. Suzlon was required to amend its code of conduct in line with amendments in insider trading regulations. However, the Code of Conduct was implemented only on February 4th, 2011, more than 2 years after the notification."
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(source, summ)
print(scores)

INFO:absl:Using default tokenizer.


{'rouge1': Score(precision=1.0, recall=0.1089588377723971, fmeasure=0.1965065502183406), 'rouge2': Score(precision=0.7954545454545454, recall=0.08495145631067962, fmeasure=0.15350877192982457), 'rougeL': Score(precision=0.8444444444444444, recall=0.09200968523002422, fmeasure=0.165938864628821)}


In [126]:
loc = ["investor", "defense", "ao"]

count = 0
flag = 0

# rouge_list = [[[], [], []], [[], [], []], [[], [], []]]
rouge_list = [[], [], []]

for i in data.iterrows():
    count+=1
    if i[1][0]=="1315460000000_":
        flag = 1
    for j in range(len(loc)):
        if loc[j]=="investor":
            if not flag:
                article = str(i[1][1]) + " " + str(i[1][3])
            else:
                article = str(i[1][1]) + " " + str(i[1][2]) + " " + str(i[1][3])
        elif loc[j]=="defense":
            article = str(i[1][1]) + " " + str(i[1][2]) + " " + str(i[1][4]) + " " + str(i[1][3])
        elif loc[j]=="ao":
            article = str(i[1][1]) + " " + str(i[1][2]) + " " + str(i[1][5])
            if not flag:
                article += " " + str(i[1][3])
        article = re.sub(" +", " ", article)
        article = re.sub(" rs\. ", " rs ", article)
        article = re.sub(" Rs\. ", " Rs ", article)
        article = re.sub(" Mrs\. ", " Mrs", article)
        article = re.sub(" Mr\. ", " Mr", article)
        article = re.sub(" Ltd\. ", " Ltd ", article)
        with open("./annot_data/" + str(loc[j]) + "/" + str(i[1][0]) + ".txt", "w+", encoding="utf-8") as f:
            f.write(re.sub("\. ", ".\n", article))
            f.close()
        pre = article.split(". ")
        out = ""
        for x in range(0, len(pre), 3):
            inputs = tokenizer([" ".join(pre[x:x+3])], max_length=max_length, return_tensors="pt", truncation=True)
            summary_ids = model.generate(inputs["input_ids"])
            curr = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
            curr = curr.replace(u'\xa0', u' ')
            curr = re.sub(" +", " ", curr)
            curr = re.sub(" Rs\. ", " Rs ", curr)
            curr = re.sub(" rs\. ", " rs ", curr)
            curr = re.sub(" Mrs\. ", " Mrs ", curr)
            curr = re.sub(" Mr\. ", " Mr ", curr)
            curr = re.sub(" Ltd\. ", " Ltd ", curr)
            if len(pre)-x<3:
                curr = ". ".join(curr.split(". ")[:len(pre)-x])
            out += " " + curr
        out = out.replace(u'\xa0', u' ')
        summ = re.sub(" +", " ", out)
        with open("./annot_out/" + str(loc[j]) + "/out_" + str(i[1][0]) + ".txt", "w+", encoding="utf-8") as f:
            f.write(re.sub("\. ", ".\n", summ))
            f.close()
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        scores = scorer.score(article, summ)
        rouge_list[j].append(scores)
        print(count, loc[j], scores)

INFO:absl:Using default tokenizer.


1 investor {'rouge1': Score(precision=0.9722222222222222, recall=0.4605263157894737, fmeasure=0.625), 'rouge2': Score(precision=0.8, recall=0.37333333333333335, fmeasure=0.5090909090909091), 'rougeL': Score(precision=0.8888888888888888, recall=0.42105263157894735, fmeasure=0.5714285714285714)}


INFO:absl:Using default tokenizer.


1 defense {'rouge1': Score(precision=0.9615384615384616, recall=0.3424657534246575, fmeasure=0.5050505050505051), 'rouge2': Score(precision=0.7142857142857143, recall=0.25229357798165136, fmeasure=0.3728813559322034), 'rougeL': Score(precision=0.7692307692307693, recall=0.273972602739726, fmeasure=0.404040404040404)}


INFO:absl:Using default tokenizer.


...

33 investor {'rouge1': Score(precision=0.9152542372881356, recall=0.4153846153846154, fmeasure=0.5714285714285714), 'rouge2': Score(precision=0.6724137931034483, recall=0.3023255813953488, fmeasure=0.41711229946524064), 'rougeL': Score(precision=0.8813559322033898, recall=0.4, fmeasure=0.5502645502645502)}


INFO:absl:Using default tokenizer.


33 defense {'rouge1': Score(precision=0.8484848484848485, recall=0.42748091603053434, fmeasure=0.5685279187817258), 'rouge2': Score(precision=0.6153846153846154, recall=0.3076923076923077, fmeasure=0.4102564102564103), 'rougeL': Score(precision=0.803030303030303, recall=0.40458015267175573, fmeasure=0.5380710659898478)}


INFO:absl:Using default tokenizer.


33 ao {'rouge1': Score(precision=0.9154929577464789, recall=0.38922155688622756, fmeasure=0.546218487394958), 'rouge2': Score(precision=0.6714285714285714, recall=0.28313253012048195, fmeasure=0.3983050847457627), 'rougeL': Score(precision=0.8732394366197183, recall=0.3712574850299401, fmeasure=0.5210084033613445)}


In [167]:
loc = ["investor", "defense", "ao"]

rouge_list = np.zeros(3)

count = 0

for i in data.iterrows():
    count +=1 
    for j in range(len(loc)):
        if loc[j]=="investor":
            ref = str(i[1][6])
        elif loc[j]=="defense":
            ref = str(i[1][7])
        else:
            ref = str(i[1][8])
        with open("./annot_ref/"+loc[j]+"/ref_"+str(i[1][0])+".txt", "w+", encoding="utf-8") as f:
            f.write(ref)
            f.close()
        # with open("./pipe_annot_out/"+loc[j]+"/out_"+str(i[1][0])+".txt", encoding="utf-8") as f:
        #     summ = f.read()
        #     summ = " ".join(summ.split("\n"))
        # scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        # scores = scorer.score(summ, ref)
        # rouge_list[0]+=scores['rouge1'][2]
        # rouge_list[1]+=scores['rouge2'][2]
        # rouge_list[2]+=scores['rougeL'][2]
        # print(count, loc[j], scorer.score(summ, ref), scorer.score(ref, summ))

# pipe_annot_out [0.45419966 0.27147896 0.34998784]
# annot_out [0.40752874 0.24601638 0.32188257]

In [163]:
print(rouge_list/99)

[0.45419966 0.27147896 0.34998784]


In [173]:
loc = ["investor", "defense", "ao"]

count = 0
flag = 0

# rouge_list = [[[], [], []], [[], [], []], [[], [], []]]
rouge_list_1 = np.zeros(3)

for i in data.iterrows():
    count+=1
    if i[1][0]=="1315460000000_":
        flag = 1
    for j in range(len(loc)):
        if loc[j]=="investor":
            if not flag:
                article = str(i[1][1]) + " " + str(i[1][3])
            else:
                article = str(i[1][1]) + " " + str(i[1][2]) + " " + str(i[1][3])
        elif loc[j]=="defense":
            article = str(i[1][1]) + " " + str(i[1][2]) + " " + str(i[1][4]) + " " + str(i[1][3])
        elif loc[j]=="ao":
            article = str(i[1][1]) + " " + str(i[1][2]) + " " + str(i[1][5])
            if not flag:
                article += " " + str(i[1][3])
        article = re.sub(" +", " ", article)
        article = re.sub(" rs\. ", " rs ", article)
        article = re.sub(" Rs\. ", " Rs ", article)
        article = re.sub(" Mrs\. ", " Mrs", article)
        article = re.sub(" Mr\. ", " Mr", article)
        article = re.sub(" Ltd\. ", " Ltd ", article)
        # with open("./annot_data/" + str(loc[j]) + "/" + str(i[1][0]) + ".txt", "w+", encoding="utf-8") as f:
        #     f.write(re.sub("\. ", ".\n", article))
        #     f.close()
        pre = article.split(". ")
        out = ""
        inputs = tokenizer([article], max_length=max_length, return_tensors="pt", truncation=True)
        summary_ids = model.generate(inputs["input_ids"])
        out = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
        out = out.replace(u'\xa0', u' ')
        summ = re.sub(" +", " ", out)
        summ = re.sub(" Rs\. ", " Rs ", summ)
        summ = re.sub(" rs\. ", " rs ", summ)
        summ = re.sub(" Mrs\. ", " Mrs ", summ)
        summ = re.sub(" Mr\. ", " Mr ", summ)
        summ = re.sub(" Ltd\. ", " Ltd ", summ)
        with open("./bart_annot_out/" + str(loc[j]) + "/out_" + str(i[1][0]) + ".txt", "w+", encoding="utf-8") as f:
            f.write(re.sub("\. ", ".\n", summ))
            f.close()
        with open("./annot_ref/"+loc[j]+"/ref_"+str(i[1][0])+".txt", encoding="utf-8") as f:
            ref = f.read()
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        scores = scorer.score(ref, summ)
        rouge_list_1[0]+=scores['rouge1'][2]
        rouge_list_1[1]+=scores['rouge2'][2]
        rouge_list_1[2]+=scores['rougeL'][2]
        print(count, loc[j], scores)

INFO:absl:Using default tokenizer.


1 investor {'rouge1': Score(precision=1.0, recall=0.13043478260869565, fmeasure=0.23076923076923078), 'rouge2': Score(precision=1.0, recall=0.11764705882352941, fmeasure=0.21052631578947367), 'rougeL': Score(precision=1.0, recall=0.13043478260869565, fmeasure=0.23076923076923078)}


INFO:absl:Using default tokenizer.


1 defense {'rouge1': Score(precision=0.9, recall=0.04864864864864865, fmeasure=0.09230769230769231), 'rouge2': Score(precision=0.8888888888888888, recall=0.043478260869565216, fmeasure=0.08290155440414508), 'rougeL': Score(precision=0.9, recall=0.04864864864864865, fmeasure=0.09230769230769231)}


INFO:absl:Using default tokenizer.


...

33 investor {'rouge1': Score(precision=0.36363636363636365, recall=0.07547169811320754, fmeasure=0.125), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.2727272727272727, recall=0.05660377358490566, fmeasure=0.09375)}


INFO:absl:Using default tokenizer.


33 defense {'rouge1': Score(precision=0.36363636363636365, recall=0.07407407407407407, fmeasure=0.12307692307692307), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.2727272727272727, recall=0.05555555555555555, fmeasure=0.0923076923076923)}


INFO:absl:Using default tokenizer.


33 ao {'rouge1': Score(precision=0.36363636363636365, recall=0.05714285714285714, fmeasure=0.09876543209876543), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.2727272727272727, recall=0.04285714285714286, fmeasure=0.07407407407407407)}


In [137]:
#brio [0.4013672  0.33043133 0.37608726]
#piped brio 0.5131486598504263 0.4041381764501251 0.4593964335064585

In [136]:
print(rouge_list_1/99)

[0.4013672  0.33043133 0.37608726]


In [132]:
r1 = np.zeros(3)
r2 = np.zeros(3)
rl = np.zeros(3)

for i in range(3):
    for j in rouge_list[i]:
        r1[i]+=j['rouge1'][2]
        r2[i]+=j['rouge2'][2]
        rl[i]+=j['rougeL'][2]

r1 = np.sum(r1/33)/3
r2 = np.sum(r2/33)/3
rl = np.sum(rl/33)/3

print(r1, r2, rl)

0.5131486598504263 0.4041381764501251 0.4593964335064585


In [147]:
summ =  "Venus Power Ventures India Limited's scrip is listed at the Bombay Stock Exchange. BSE on the basis of an internal alert conducted an investigation in the said Scrip for the period from 01 04 2012 to 31 01 2013. There was a decrease in the holding of Ravi Chakravarthy Noticee. I impose a penalty of 50000 Rupees fifty thousand on the noticee Viz. for failing to make disclosures to the company and stock exchanges in the September 2012 quarter. The noticee admitted that he did not make any disclosures. the penalty is in line with the regulations of the SEBI."
summ_ind_tokens = [word_tokenize(i) for i in summ.split(". ")]
# print(summ_ind_tokens)
id2word = corpora.Dictionary(summ_ind_tokens)
corpus = [id2word.doc2bow(text) for text in summ_ind_tokens]
# cm = CoherenceModel(topics=summ_ind_tokens, corpus=corpus, dictionary=id2word, coherence='u_mass')
cm = CoherenceModel(topics=summ_ind_tokens, corpus=common_corpus, dictionary=common_dictionary, coherence='u_mass')
print(cm.get_coherence())
#-0.9009510160545718
#-0.7023449409828717
#-0.344044747903167

KeyError: 'Venus'

In [182]:
pipe_metrics = [[[], [], [], []], [[], [], [], []], [[], [], [], []]]

loc = ["investor", "defense", "ao"]

itcount = 0

for j in range(len(loc)):
    for i in os.listdir("./gen_data/" + str(loc[j]) + "/"):
        if itcount==25:
            break
        itcount+=1
        with open("./gen_data/" + str(loc[j]) + "/" + str(i), encoding="utf-8") as f:
            text = f.read()
            f.close()
        pre = text.split("\n")
        if len(pre) < 2:
            continue
        article = " ".join(pre)
        out = ""
        for x in range(0, len(pre), 3):
            inputs = tokenizer([" ".join(pre[x:x+3])], max_length=max_length, return_tensors="pt", truncation=True)
            summary_ids = model.generate(inputs["input_ids"])
            curr = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
            curr = curr.replace(u'\xa0', u' ')
            curr = re.sub(" +", " ", curr)
            curr = re.sub(" Rs\. ", " Rs ", curr)
            curr = re.sub(" rs\. ", " rs ", curr)
            curr = re.sub(" Mrs\. ", " Mrs ", curr)
            curr = re.sub(" Mr\. ", " Mr ", curr)
            curr = re.sub(" Ltd\. ", " Ltd ", curr)
            if len(pre)-x<3:
                curr = ". ".join(curr.split(". ")[:len(pre)-x])
            out += " " + curr
        out = out.replace(u'\xa0', u' ')
        summ = re.sub(" +", " ", out)
        # print(summ)
        with open("./pipe_gen_out/" + str(loc[j]) + "/out_" + str(i), "w+", encoding="utf-8") as f:
            f.write(summ)
            f.close()
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        scores = scorer.score(article, summ)
        print(scores)
        # source_sbert = smodel.encode(article, convert_to_tensor=True)
        # summ_sbert = smodel.encode(summ, convert_to_tensor=True)
        # try:
        #     pipe_metrics[j][0].append(util.cos_sim(source_sbert, summ_sbert))
        # except:
        #     pipe_metrics[j][0].append(0)
        # source_sbert = np.array(source_sbert.cpu())
        # summ_sbert = np.array(summ_sbert.cpu())
        # source_tokens = word_tokenize(article)
        # summ_tokens = word_tokenize(summ)
        # # print(len(summ_tokens), len(source_tokens))
        # try:
        #     pipe_metrics[j][1].append(len(summ_tokens)/len(source_tokens))
        # except:
        #     pipe_metrics[j][1].append(0)
        # count = 0
        # sum = 0
        # summlist = summ.split(". ")
        # for ind1, m in enumerate(summlist):
        #     for ind2, n in enumerate(summlist[ind1+1:]):
        #         sbert_m = smodel.encode(m, convert_to_tensor=True)
        #         sbert_n = smodel.encode(n, convert_to_tensor=True)
        #         count+=1
        #         sum+=util.cos_sim(sbert_m, sbert_n)[0][0]
        # try:
        #     pipe_metrics[j][2].append(sum/count)
        # except: 
        #     pipe_metrics[j][2].append(0)
        # summ_ind_tokens = [word_tokenize(i) for i in summ.split(". ")]
        # try:
        #     id2word = corpora.Dictionary(summ_ind_tokens)
        #     corpus = [id2word.doc2bow(text) for text in summ_ind_tokens]
        #     cm = CoherenceModel(topics=summ_ind_tokens, corpus=corpus, dictionary=id2word, coherence='u_mass')
        # except:
        #     id2word = corpora.Dictionary([source_tokens, summ_tokens])
        #     corpus = [id2word.doc2bow(text) for text in [source_tokens, summ_tokens]]
        #     cm = CoherenceModel(topics=[source_tokens, summ_tokens], corpus=corpus, dictionary=id2word, coherence='u_mass')
        # pipe_metrics[j][3].append(cm.get_coherence())
        # print(itcount, np.sum(np.array(pipe_metrics[j]), axis=1)/np.shape(pipe_metrics[j])[1])

# 0.9065 0.400 0.375

KeyboardInterrupt: 

In [ ]:
ref = ""
summ = ""

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(ref, summ)

In [94]:
metrics = [[[], [], [], []], [[], [], [], []], [[], [], [], []]]

loc = ["investor", "defense", "ao"]

itcount = 0

for j in range(len(loc)):
    for i in tqdm(os.listdir("./gen_data/" + loc[j] + "/")):
        if itcount==100:
            break
        itcount+=1
        with open("./gen_data/" + loc[j] + "/" + i, encoding="utf-8") as f:
            text = f.read()
            f.close()
        pre= text.split("\n")
        article = " ".join(pre)
        inputs = tokenizer([article], max_length=max_length, return_tensors="pt", truncation=True)
        summary_ids = model.generate(inputs["input_ids"])
        out = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
        out = out.replace(u'\xa0', u' ')
        summ = re.sub(" +", " ", out)
        with open("./gen_out/" + loc[j] + "/out_" + i, "w+", encoding="utf-8") as f:
            f.write(re.sub(". ", ".\n", summ))
            f.close()
        source_sbert = smodel.encode(article, convert_to_tensor=True)
        summ_sbert = smodel.encode(summ, convert_to_tensor=True)
        try:
            metrics[j][0].append(util.cos_sim(source_sbert, summ_sbert))
        except:
            metrics[j][0].append(0)
        source_sbert = np.array(source_sbert.cpu())
        summ_sbert = np.array(summ_sbert.cpu())
        source_tokens = word_tokenize(article)
        summ_tokens = word_tokenize(summ)
        # print(len(summ_tokens), len(source_tokens))
        try:
            metrics[j][1].append(len(summ_tokens)/len(source_tokens))
        except:
            metrics[j][1].append(0)
        count = 0
        sum = 0
        summlist = summ.split(". ")
        for ind1, m in enumerate(summlist):
            for ind2, n in enumerate(summlist[ind1+1:]):
                sbert_m = smodel.encode(m, convert_to_tensor=True)
                sbert_n = smodel.encode(n, convert_to_tensor=True)
                count+=1
                sum+=util.cos_sim(sbert_m, sbert_n)[0][0]
        try:
            metrics[j][2].append(sum/count)
        except: 
            metrics[j][2].append(0)
        summ_ind_tokens = [word_tokenize(i) for i in summ.split(". ")]
        try:
            id2word = corpora.Dictionary(summ_ind_tokens)
            corpus = [id2word.doc2bow(text) for text in summ_ind_tokens]
            cm = CoherenceModel(topics=summ_ind_tokens, corpus=corpus, dictionary=id2word, coherence='u_mass')
        except:
            id2word = corpora.Dictionary([source_tokens, summ_tokens])
            corpus = [id2word.doc2bow(text) for text in [source_tokens, summ_tokens]]
            cm = CoherenceModel(topics=[source_tokens, summ_tokens], corpus=corpus, dictionary=id2word, coherence='u_mass')
        metrics[j][3].append(cm.get_coherence())
        print(itcount, np.sum(np.array(metrics[j]), axis=1)/np.shape(metrics[j])[1])

# [tensor([[0.7936]]) 0.08259719412351119 tensor(0.3949) nan]

  0%|          | 0/1495 [00:00<?, ?it/s]<ipython-input-94-eca4c135c937>:62: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  print(itcount, np.sum(np.array(metrics[j]), axis=1)/np.shape(metrics[j])[1])
  0%|          | 1/1495 [00:12<5:11:38, 12.52s/it]

1 [ 0.67055345  0.14933333  0.30950633 -0.0638425 ]


<ipython-input-94-eca4c135c937>:62: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(itcount, np.sum(np.array(metrics[j]), axis=1)/np.shape(metrics[j])[1])
  0%|          | 2/1495 [00:27<5:26:41, 13.13s/it]

2 [tensor([[0.7616]]) 0.09505941591137966 tensor(0.4269)
 -0.051986038540701114]


  0%|          | 3/1495 [00:43<5:52:43, 14.18s/it]

...

96 [tensor([[0.7945]]) 0.08254573685408807 tensor(0.3938) nan]


  6%|▋         | 97/1495 [23:19<6:14:46, 16.09s/it]

97 [tensor([[0.7936]]) 0.08259719412351119 tensor(0.3949) nan]


  6%|▋         | 97/1495 [23:27<5:38:09, 14.51s/it]


KeyboardInterrupt: 

In [31]:
source = "Securities and Exchange Board of India (hereinafter referred to as ‘SEBI’) has conducted investigation into the alleged irregularity in the shares of Shelter Infra Projects Ltd., formerly known as CCAP Ltd (hereinafter referred to as ‘SIPL’) and into the possible violations of the SEBI Act, 1992 (hereinafter referred to as the Act) and the various Regulations made there under for the period from April 01, 2009 to September 22, 2009. The Investigation, inter alia, revealed that the Board of Directors of SIPL approved a Shares Purchase Agreement at its meeting held on July 30, 2009 to be executed between the SIPL and the proposed acquirers on July 31, 2009 for the sale of the issued and subscribed equity share capital of the company held by the erstwhile promoters, as also change in management of the company. On August 07, 2009 the managers to the offer on behalf of the acquirer issued a public announcement to the equity shareholders of SIPL. However, SIPL has No introduced any trading window during the above public announcement. Shri K. L. Surana, (Noticee No 1) was the Compliance Officer and the entities mentioned above at serial No. 2 to 8 were the directors of SIPL (herein after referred to as Noticee Nos.1 to 8 respectively and collectively the Noticees) and were present at the Board Meeting held on July 30, 2009. SEBI has therefore initiated adjudication proceedings under the Act against the Noticees to inquire into and adjudge the alleged violations of Regulation 12 (1) read with Clause 1.2 of Schedule I, Part A of SEBI (Prohibition of Insider Trading) Regulations, 1992 (hereinafter referred to as PIT Regulations). SIPL vide letter dated October 13, 2012 submitted a copy of the Death Certificate of Noticee No. 1 issued by the Department of Health and Welfare, Government of West Bengal dated May 22, 2012. I find that the Noticee No 1 was the compliance officer, and Noticee Nos. 2 to 8 were the Board of Director of SIPL and were present in the Board Meeting held on July 30, 2009 when a Shares Purchase Agreement for the sale of 35.50 % of the issued and subscribed equity share capital of SIPL held by the erstwhile promoters to M s. Ramayana Promoters Pvt. Ltd. (Acquirer) as also change in control of the management of the company was approved. The information that M s. Ramayana Promoters Pvt. Ltd. was taking over the Company became public only on August 7, 2009. I note that SIPL confirmed that no trading window was introduced during the above public announcement vide its letter dated April 04, 2010. Further, no separate letter was issued to the Board members instructing them of the commencement for the closing of the trading window and SIPL had left its trading window open during period of public announcement (ten days before and two days after the public announcement) in contravention to the Code of Conduct for prevention of insider trading followed by SIPL. It is an admitted fact that mere circulation of code of conduct to all the Board members during March end does No tantamount to a communication that the trading window had to be closed before the public announcement.  A formal communication indicating the trading window to be closed ought to have been made before the public announcement by the Compliance Officer. "
summ = "SEBI has conducted investigation into alleged irregularity in the shares of Shelter Infra Projects Ltd., formerly known as CCAP Ltd. Investigation revealed that the Board of Directors of SIPL approved a Shares Purchase Agreement in 2009. No trading window was introduced during the public announcement of the agreement."

In [34]:
source_sbert = model.encode(source, convert_to_tensor=True)
summ_sbert = model.encode(summ, convert_to_tensor=True)
print("Semantic similarity", util.cos_sim(source_sbert, summ_sbert))
source_sbert = np.array(source_sbert.cpu())
summ_sbert = np.array(summ_sbert.cpu())

Semantic similarity tensor([[0.8094]])


In [35]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\e_sai\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [36]:
source_tokens = word_tokenize(source)
summ_tokens = word_tokenize(summ)
print(len(summ_tokens), len(source_tokens))
print("Compression", len(summ_tokens)/len(source_tokens))

51 620
Compression 0.08225806451612903


In [45]:
import re
summ = summ.replace(u'\xa0', u' ')
summ = re.sub(" +", " ", summ)
summ = re.sub(" Rs\. ", " Rs.", summ)
summ = re.sub(" rs\. ", " rs.", summ)
summlist = summ.split(". ")

count = 0
sum = 0
for ind1, i in enumerate(summlist):
  for ind2, j in enumerate(summlist[ind1+1:]):
    sbert_i = model.encode(i, convert_to_tensor=True)
    sbert_j = model.encode(j, convert_to_tensor=True)
    count+=1
    sum+=util.cos_sim(sbert_i, sbert_j)[0][0]

print(count, "Redundancy", sum/count)

3 Redundancy tensor(0.2783)


In [55]:
id2word = corpora.Dictionary([source_tokens])
corpus = [id2word.doc2bow(text) for text in [source_tokens]]
cm = CoherenceModel(topics=[source_tokens, summ_tokens], corpus=corpus, dictionary=id2word, coherence='u_mass')
coherence = cm.get_coherence()
print(coherence)

1.0000889005818406e-12


In [38]:
cm = CoherenceModel(topics=[source_tokens, summ_tokens[:512]], corpus=common_corpus, dictionary=common_dictionary, coherence='u_mass')
coherence = cm.get_coherence()

KeyError: 'Securities'

In [29]:
for i in tqdm(os.listdir("./annot/")):
    if i[-4:]==".csv":
        data = pd.read_csv("./annot/"+i)
        investor = '\n'.join(data[data['Label'].isin(['material fact', 'penalty'])]['Sentence'].tolist())
        defense = '\n'.join(data[data['Label'].isin(['material fact', 'defendant claim', 'issues framed'])]['Sentence'].tolist())
        ao = '\n'.join(data[data['Label'].isin(['material fact', 'rel fact', 'penalty'])]['Sentence'].tolist())
        investor = re.sub(" +", " ", investor)
        defense = re.sub(" +", " ", defense)
        ao = re.sub(" +", " ", ao)
        with open("./gen_data/investor/"+i[:-4]+"_investor.txt", "w+", encoding="utf-8") as f:
            f.write(investor)
            f.close()
        with open("./gen_data/defense/"+i[:-4]+"_defense.txt", "w+", encoding="utf-8") as f:
            f.write(defense)
            f.close()
        with open("./gen_data/ao/"+i[:-4]+"_ao.txt", "w+", encoding="utf-8") as f:
            f.write(ao)
            f.close()

100%|██████████| 2990/2990 [00:12<00:00, 235.79it/s]
